In [1]:
import pandas as pd
import polars as pl

import numpy as np

import lectura_archivos as fr

# Analisis de Cantidad de Traqueostomias por Servicio de Salud

- En este apartado se quiere ver la cantidad de traqueostomias que se hicieron por Servicio de Salud.
Este analisis se realiza por anio.

## Carga de Archivos de GRD

In [2]:
dfp = fr.leer_grd()


In [3]:
df = dfp.to_pandas()


## Analisis de Traqueostomia

In [5]:
procedimientos = df.loc[:, "PROCEDIMIENTO1":"PROCEDIMIENTO30"]
contiene_nuestros_proc = procedimientos.isin(["31.1", "31.29"]).sum(axis=1).astype(bool)
proc_nacionales = df[contiene_nuestros_proc]


In [6]:
metricas = (
    proc_nacionales.groupby(["ANIO_EGRESO", "SERVICIO_SALUD"])
    .agg(
        n_egresos=("COD_HOSPITAL", "count"),
        peso_medio=("IR_29301_PESO", "mean"),
        estancia_media=("ESTANCIA", "mean"),
    )
    .sort_values(["ANIO_EGRESO", "n_egresos"], ascending=False)
)

In [7]:
metricas.to_excel("output/resumen_traqueostomia_hist.xlsx")

# Analisis de Prestaciones por Diagnostico

- En este apartado se quiere ver la cantidad de prestaciones que se realizan por cada diagnostico
principal.

- El resultado que se quiere obtener es parecido al siguiente:

|Año|Diagnostico|Resumen Procedimientos|
|---|-----------|----------------------|
|2019|"I25.1"|{"45.23": 20, "73.59": 10}|
|2020|"I25.1"|{"45.23": 7, "73.59": 3}|

## Analisis Exploratorio

### Obtener conteo de diagnosticos en el Torax

In [4]:
torax = dfp.filter(pl.col("COD_HOSPITAL") == 112103)
torax.groupby(["DIAGNOSTICO1"]).agg(pl.count()).sort(by="count", descending=True)


DIAGNOSTICO1,count
str,u32
"""I25.1""",1366
"""Z51.1""",811
"""U07.1""",706
"""Z51.4""",595
"""Z13.6""",489
"""Z29.1""",394
"""C34.1""",367
"""I35.0""",365
"""Z45.0""",322


- De estos resultados, se puede ver que el diagnostico Z51.4 es el diagnostico mas frecuente
entre 2019 y 2021. Tuvo 1366 egresos.

- A modo de ejemplo, se quiere ver el dataframe completo del diagnostico Z51.4, y analizar
los procedimientos que tiene incluidos.

### Analisis de conteo de procedimientos

### Forma 1

In [5]:
from functools import reduce

i251 = torax.filter(pl.col("DIAGNOSTICO1") == "I25.1").to_pandas()
listas_procedimientos = (
    i251.loc[:, "PROCEDIMIENTO1":"PROCEDIMIENTO30"].fillna("").agg("-".join, axis=1).str.split("-")
)


sumado = reduce(lambda x, y: x + y, listas_procedimientos.values)

### Forma 2

In [6]:
cambiada = pd.melt(
    i251, id_vars="DIAGNOSTICO1", value_vars=[f"PROCEDIMIENTO{i}" for i in range(1, 31)]
)
cambiada["value"].value_counts()


99.29    1196
99.19    1125
89.54    1035
93.96     930
37.22     806
         ... 
78.61       1
39.98       1
99.10       1
37.99       1
37.87       1
Name: value, Length: 257, dtype: int64

## Analisis Completo

- Se quiere ver la cantida dde procedimientos hechos por posicion de procedimiento. O sea,
en el "PROCEDIMIENTO1" que procedimientos se ingresaron, y cuales son sus frecuencias ordenadas
de mayor a menor?

- Tambien se quiere ver la frecuencia global. O sea, para el diagnostico X, cuales son los
procedimientos que mas se le realizan, independiente de la posicion del procedimiento.

#### Cambio de nombre de columnas PROCEDIMIENTO

- Esto se realiza, ya que permite ordenar estas columnas al utilizar .value_counts() en la
proxima celda.

In [7]:
import string

procedimientos = [f"PROCEDIMIENTO{i}" for i in range(1, 31)]
letras = list(string.ascii_uppercase) + ["a", "b", "c", "d"]
codificacion = dict(zip(procedimientos, letras))
codificacion_reversa = dict(zip(letras, procedimientos))

torax_pandas = torax.to_pandas()
long_prestaciones = pd.melt(
    torax_pandas,
    id_vars=["ANIO_EGRESO", "DIAGNOSTICO1"],
    value_vars=procedimientos,
)

long_prestaciones["variable"] = long_prestaciones["variable"].replace(codificacion)

In [28]:
resumen_prestaciones_global = (
    long_prestaciones.groupby(["ANIO_EGRESO", "DIAGNOSTICO1"])["value"]
    .value_counts()
    .reset_index(name="n_procedimientos")
)

por_procedimiento = (
    long_prestaciones.groupby(["ANIO_EGRESO", "DIAGNOSTICO1", "variable"])["value"]
    .value_counts()
    .reset_index(name="n_procedimientos")
)

por_procedimiento["variable"] = por_procedimiento["variable"].replace(codificacion_reversa)
por_procedimiento = por_procedimiento.rename({"value": "codigo_procedimiento"}, axis=1)


In [29]:
resumen_prestaciones_global.to_excel("output/prestaciones_globales.xlsx")
por_procedimiento.to_excel("output/prestaciones_por_procedimiento.xlsx")

### Obtener Cantidad de Egresos

In [30]:
cantidad_egresos = (
    long_prestaciones.query('variable == "A"').groupby("ANIO_EGRESO").DIAGNOSTICO1.value_counts()
).reset_index(name="n_egresos")

cantidad_egresos_totales = long_prestaciones.query('variable == "A"').DIAGNOSTICO1.value_counts()
display(cantidad_egresos_totales)

I25.1    1366
Z51.1     811
U07.1     706
Z51.4     595
Z13.6     489
         ... 
T86.0       1
N62         1
H44.0       1
G53.0       1
C39.8       1
Name: DIAGNOSTICO1, Length: 568, dtype: int64

- Se puede observar que este metodo de calculo permite obtener la cantidad de egresos de 
[forma similar a lo visto anteriormente](#obtener-conteo-de-diagnosticos-en-el-torax)

In [33]:
procedimientos_egresos = pd.merge(
    por_procedimiento, cantidad_egresos, how="inner", on=["ANIO_EGRESO", "DIAGNOSTICO1"]
)